# Spark Optimization for Advances Users

<a id="contents"></a>

## Contents

- [RDD](#rdd)
- [Partitions](#partitions)
- [Joins](#joins)
- [Serialisation](#dataserialisation)
- [UDF](#udf)
- [Analyse Execution plan](#executionplan)
- [Data Skew](#data_skew)
- [Cache](#cache)
- [Storage](#storage)
- [JVM](#jvm)
- [Monitoring](#monitoring)
- [Misc](#misc)

<a id='rdd'></a>

## RDD


* Avoid `groupBy` and prefer `reduceBy`

**groupByKey**

`groupByKey` can cause out of disk problems as data is sent over the network and collected on the reduce workers.

Syntax:

```scala
sparkContext.textFile("hdfs://")
                    .flatMap(line => line.split(" ") )
                    .map(word => (word,1))
                    .groupByKey()
                    .map((x,y) => (x,sum(y)))
```
Eg:

```scala
  val a = sc.parallelize(List("dog", "cat", "owl", "gnu", "ant"), 2)
  val b = a.map(x => (x.length, x))
  b.reduceByKey(_ + _).collect
  // b.groupByKey().mapValues(_.mkString("")).collect()
  // Array[(Int, String)] = Array((3,dogcatowlgnuant))  
```

**reduceByKey**

Data are combined at each partition, only one output for one key at each partition to send over the network. reduceByKey required combining all your values into another value with the exact same type

Syntax:

```scala
sparkContext.textFile("hdfs://")
                    .flatMap(line => line.split(" "))
                    .map(word => (word,1))
                    .reduceByKey((x,y)=> (x+y))
```

```scala
  val a = sc.parallelize(List("dog", "tiger", "lion", "cat", "panther", "eagle"), 2)
  val b = a.map(x => (x.length, x))
  b.reduceByKey(_ + _).collect
  //// b.groupByKey().mapValues(_.mkString("")).collect()
  // Array[(Int, String)] = Array((4,lion), (3,dogcat), (7,panther), (5,tigereagle))
```

**aggregateByKey:**

same as reduceByKey, which takes an initial value.

3 parameters as input i. initial value ii. Combiner logic iii. sequence op logic

Eg:

```scala
val keysWithValuesList = Array("foo=A", "foo=A", "foo=A", "foo=A", "foo=B", "bar=C", "bar=D", "bar=D")
    val data = sc.parallelize(keysWithValuesList)
    //Create key value pairs
    val kv = data.map(_.split("=")).map(v => (v(0), v(1))).cache()
    val initialCount = 0;
    val addToCounts = (n: Int, v: String) => n + 1
    val sumPartitionCounts = (p1: Int, p2: Int) => p1 + p2
    val countByKey = kv.aggregateByKey(initialCount)(addToCounts, sumPartitionCounts)
//ouput: Aggregate By Key sum Results bar -> 3 foo -> 5
```

**combineByKey:**

3 parameters as input

Initial value: unlike aggregateByKey, need not pass constant always, we can pass a function that will return a new value.
merging function
combine function

```scala
Eg:
val result = rdd.combineByKey(
                        (v) => (v,1),
                        ( (acc:(Int,Int),v) => acc._1 +v , acc._2 +1 ) ,
                        ( acc1:(Int,Int),acc2:(Int,Int) => (acc1._1+acc2._1) , (acc1._2+acc2._2)) 
                        ).map( { case (k,v) => (k,v._1/v._2.toDouble) })
result.collect.foreach(println)
```

[<< Contents](#contents)

<a id='partitions'></a>


----------------------------------------------------------------------------------------------------------------------------------------------------


## Partitions - That Detemines the parallelism

  * Avoid spills and utilize all the cores
  * At input level
    * `spark.default.parallelism`(don't use)
    * `spark.sql.files.maxPartitionBytes`
      * Use default partition size **128MB** , unless
        * Increase parallelism
        * Heavily nested/repeated data
        * Generating data
        * Source structure is not optimal
        * UDFs
      * ![](images/paritionfilesize.png)
  * At shuffle level
    * `sparl.sql.shuffle.partitions`
    * Default maximum size is **2GB**

  - At ouput level

    - Coalesce to shrink number of partitions

    - Repartition to increase the number of partitions

    - ```python
      df.write.option("maxRecordsPerFile", n)
      df.coalesce(n).write...
      df.repartition(n).write...
      df.repartition(n, [colA, ...]).write...
      spark.sql.shuffle.partitions(n)
      df.localCheckpoint(...).repartition(n).write...
      df.localCheckpoint(...).coalesce(n).write...
      ```

    - ```
      Write Data Size = 14.7GB
      Desired File Size = 1500MB
      Max write stage parallelism = 10
      # Say we have 96 cores
      96 – 10 == 86 cores idle during write
      
      14.7/96 cores = ~ 160 MB of parition size
      spark.sql.files.maxPartitionBytes = 160 * 1024 * 1024
      So now we use all 96 cores
      ```

    - ![](images/spark_partition_file_size.png)

  - **Never rely on default partition size of `200` at any cost**

    - ```
      Say if you have data size of 210GB
      number_partitions = 210000MB / 200MB = 1050
      # irrespective of number of cores
      spark.conf.set("Spark.sql.shuffle.paritions", 1050)
      # say if the cluster has 2000 cores
      spark.conf.set("Spark.sql.shuffle.paritions", 2000)
      ```

    - ![](images/spill.png)

    - ![](images/spill_corrected.png)

[<< Contents](#contents)

<a id="joins"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## Joins

  * SortMerge Join – Both sides are lrage

  * **Broadcast DataFrame** Join when one side is small

    * Default size is 10MB
* Spark SQL uses **broadcast join** (aka **broadcast hash join**) instead of hash join to optimize join queries when the size of one side data is below [spark.sql.autoBroadcastJoinThreshold](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-properties.html#spark.sql.autoBroadcastJoinThreshold).
    * Broadcast allows to send a read-only variable cached on each node once, rather than sending a copy for all tasks. 
    * Automatic If:
        (one side < spark.sql.autoBroadcastJoinThreshold) (default 10m)
      * Risks
        - Not Enough Driver Memory
        - DF > spark.driver.maxResultSize
        - DF > Single Executor Available Working Memory
      * ![](images/broadcat_explained.png)
    
  * Requires to use Hive and its metastore for auto detection of table size
    
    ```scala
    leftDF.join(broadcast(rightDF))
    ```
    
  * **Put the Largest Dataset on the Left**
  
    * When you’re joining together two datasets where one is smaller than the other, put the larger dataset on the “Left”:
    * When Spark shuffles data for the join, it keeps the data you specified on  the left static on the executors and transfers the data you designed on  the right between the executors. If the data that’s on the right, that’s being transferred, is larger, then the serialization and transfer of  the data will take longer.
  
  ```
  val joinedDF = largerDF.leftJoin(smallerDF, largerDF("id") === smallerDF("some_id"))
  ```
    * Be aware of `null` values 
  
        * Consider sample query where we are joining on highly null columns
  
        * ![](images/null_counts.png)
  
          ```sql
          select * from  order_tbl orders left join customer_tbl  customer
                on orders.customer_id = customer.customer_id
                left join delivery_tbl delivery
                on  orders.delivery_id = delivery.delivery_id
          ```
  
        * In above query, 199/200 tasks would complete quite fast and then probably gets stuck on the last task. 
  
        * ***Reason for the above behavior:\*** Let’s say that we have asked Spark to join two DataFrames — TABLE1 and  TABLE2. When Spark executes this code, internally it performs the  default Shuffle Hash Join (Exchange hashpartitioning).
  
          ```
          +- Exchange hashpartitioning(delivery_id#22L, 400)
                         +- *(6) Project [delivery_id#22L]
          ```
  
          In this process, Spark hashes the join column and sorts it. And then it  tries to keep the records with same hashes in both partitions on the  same executor hence all the null values of the table will go to one  executor and spark gets into a continuous loop of shuffling and garbage  collection with no success.
  
      * Solution is split the tables into two parts, one with null values(i.e all the values) and without null values. 
  
      * Use table without null values for join and then union them with full data
  
      * This way null values won't participate in the join
  
      * ```sql
        CREATE TABLE order_tbl_customer_id_not_null as select * from order_tbl where customer_id is not null;
        
        CREATE TABLE order_tbl_customer_id_null as select * from order_tbl where customer_id is null;
        
        --Rewrite query
        select orders.customer_id
        from  order_tbl_customer_id_not_null orders left join customer_tbl  customer 
                 on orders.customer_id = customer.customer_id
        union all
        select ord.customer_id from order_tbl_customer_id_null ord;
        ```
        
    * Coalesce vs repartition

      In the literature, it’s often mentioned that *coalesce* should be preferred over *repartition* to reduce the number of partitions because it avoids a shuffle step in some cases.

      But ***coalesce\* has some limitations** (outside the scope of this article): it cannot increase the number of partitions and may generate skew partitions.

      Here is one case where a **repartition should be preferred**. In this case, we filter most of a dataset.

      ```
      df.doSth().coalesce(10).write(…)
      ```



      ![img](images/coalesce.png)

      The good point about *coalesce* is that it avoids a shuffle. However, all the computation is done by only 10 tasks.

      This is due to the fact that **the number of tasks depends on the number of partitions of the output of the stage**, each one computing a big bunch of data. So a maximum of 10 nodes will perform the computation.

      ```
      df.doSth().repartition(10).write(…)
      ```

      ![](images/repartition.png)

      Using *repartition* we can see that the total duration is way shorter (a few seconds  instead of 31 minutes). The filtering is done by 200 tasks, each one  working on a small subset of data. It’s also way smoother from a memory  point a view, as we can see in the graph below.

      ![](images/repartition_memory.png)
  
[<< Contents](#contents)

<a id="dataserialisation"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## Serialisation

  - It is the process of converting the in-memory object to another format  that can be used to store in a file or send over the network.
  - Two options
    - Java serialization
    - [Kryo serialization](https://github.com/EsotericSoftware/kryo)
  - Configuration : “spark.serializer” to “org.apache.spark.serializer.KyroSerializer"

```scala
 val conf = new SparkConf().setAppName(...).setMaster(...)
      .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
      .set("spark.kryoserializer.buffer.max", "128m")
      .set("spark.kryoserializer.buffer", "64m")
      .registerKryoClasses(
        Array(classOf[ArrayBuffer[String]], classOf[ListBuffer[String]])
      )
``` 
[<<< Contents](#contents)

<a id="udf"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## UDF


  * Avoid User defined Function **UDFs** and User Defined Aggregate Funtions **UDAF** 
  * Prefer in build SQL Functions where ever possible
  * Traditional UDFs cannot use Tungsten

    * Use `org.apache.spark.sql.functions`
    * Use PandasUDFs

      * Utilizes Apache Arrow
    * Use SparkR UDFs
  * https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html
  * https://fr.slideshare.net/cfregly/advanced-apache-spark-meetup-project-tungsten-nov-12-2015

[<< Contents](#contents)

<a id="executionplan"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## Analyse Execution plan

  - Stages are created when there is a need for shuffle for operations like `join` or `groupBy`

  - **Number of stages**

    * Thought the Spark engine does pretty good job of optimizing the DAGs for executions, it is also developer responsibility to keep the number of stages under a reasonable number.
    * This involves good amount of understanding of APIs and SQL query optimization

  - We use the` .explain(true)` command to show the execution plan detailing all the steps (stages) involved for a job. Here is an example:

    ![](images/explain.png)
    
[<<< Contents](#contents)

<a id="data_skew"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## Data Skew

  - Common symptoms of data skew are:
    - Frozen stages and tasks.
      - Especially with the last couple of tasks in a stage
    - Low utilization of CPU.
    - Out of memory errors.
  - Data broadcast during join operation can help with minimize the data skew side effects. Increase the `spark.sql.autoBroadcastJoinThreshold` for Spark to consider tables of bigger size. Make sure enough memory is available in driver and executors
  - Salting
    - In a SQL join operation, the join key is changed to redistribute data in an even manner so that processing for a partition does not take more  time. This technique is called salting.
    
    -  Add Column to each side with random int between 0 and `spark.sql.shuffle.partitions` – 1 to both sides
    
    - Add join clause to include join on generated column above
    
    - Drop temp columns from result
    
    - ```scala
      df.groupBy(“city”, “state”).agg(<f(x)>).orderBy(col.desc)
      val saltVal = random(0, spark.conf.get(org...shuffle.partitions) - 1)
      df.withColumn(“salt”, lit(saltVal))
      .groupBy(“city”, “state”, “salt”)
      .agg(<f(x)>)
      .drop(“salt”)
      .orderBy(col.desc)
      
      ```
      
* **Highly imbalanced dataset**

  - To quickly check if everything is ok we review the execution duration of each task and look for **heterogeneous process time**. If one of the tasks is significantly slower than the others it will  extend the overall job duration and waste the resources of the fastest  executors.

    ![](images/stage_execution_time.png)
    

- **COMPUTE STATISTICS** of Tables Before Processing

  - Before querying a series of tables, it can be helpful to tell spark to Compute the Statistics of those tables so that the Catalyst Optimizer can come  up with a better plan on how to process the tables.

  ```
  spark.sql("ANALYZE TABLE dbName.tableName COMPUTE STATISTICS")
  ```

  - In some cases, Spark doesn’t get everything it needs from just the above  broad COMPUTE STATISTICS call. It also helps to tell Spark to check  specific columns so the Catalyst Optimizer can better check those  columns. It’s recommended to COMPUTE STATISTICS for any columns that are involved in filtering and joining.

```
spark.sql("ANALYZE TABLE dbName.tableName COMPUTE STATISTICS FOR COLUMNS joinColumn, filterColumn")
```

- Avoid calling **DataFrame/RDD `count` API** unless it is absolutely necessary. 

[<<< Contents](#contents)

<a id="cache"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## Cache

* **Inappropriate use of caching**

  There is no universal answer when choosing what should be cached. Caching an intermediate result can **dramatically improve performance** and it’s tempting to cache a lot of things. However, due to Spark’s  caching strategy (in-memory then swap to disk) the cache can end up in a slightly slower storage. Also, using that storage space for caching  purposes means that it’s not available for processing. In the end,  caching might cost more than simply reading the *DataFrame*.

  In the *Storage* tab of the UI we verify the *Fraction Cached* and also look at the *Size in Memory* and *Size on Disk* distribution.

  ![](images/cacahe.png)
  
[<<< Contents](#contents)

<a id="storage"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## Storage

* **Pitfalls with AWS S3**

  - A simple renaming actually needs to copy and then delete the original file.

  - The first [workaround](https://medium.com/@subhojit20_27731/apache-spark-and-amazon-s3-gotchas-and-best-practices-a767242f3d98) when using Spark with S3 as an output it to use this specific configuration:

    ```shell
    spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version 2 spark.speculation false
    ```
    
- **Data Skipping**
  
  - **Parquet**
    
    - Columnar storage format
    - Spark can leverage **predicate push down** (i.e. pushing down the filtering closer to the data).
    - However, predicate pushdown should be used with extra care. Even if it appears in the execution plan, **it will not always be effective**. For example, a filter push-down does not work on String and Decimal data types (cf[ PARQUET-281](https://issues.apache.org/jira/browse/PARQUET-281)).
    
  - Hive Partitions
  
  - Hive **Bucketing**
  
    * Consider when there is a high hashable column values
  
    * `spark.sql.sources.bucketing.enabled=True`
  
    * Set  `hive.enforce.bucketing=false` and `hive.enforce.sorting=false`
  
    * https://www.slideshare.net/databricks/hive-bucketing-in-apache-spark-with-tejas-patil
  
    * ```scala
      df.write
        .bucketBy(numBuckets, "col1", ...)
        .sortBy("col1", ...)
        .saveAsTable("/path/table_name")
      ```
  
    * ```sql
      create table table_name(col1 INT, ...)
      	using parquet
      	CLUSTERED By (col1, ...)
      	SORTED BY(col1, ...)
      	INTO `numBuckets` BUCKETS
      ```
  
    * ![](images/bucketing.png)
    
[<<< Contents](#contents)

<a id="jvm"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## Java Garbage Collection

To more on the topic read [here](https://javarevisited.blogspot.com/2011/04/garbage-collection-in-java.html)

<img src="images/JavaGarbageCollectionheap.png" />

`spark.executor.extraJavaOptions = -XX:+UseConcMarkSweepGC -XX:+CMSClassUnloadingEnabled -XX:UseG1GC XX:InitiatingHeapOccupancyPercent=35 -XX:ConcGCThread=20`

  - Analyse the logs for the memory usage, most likely the problem would be with the data partitions.
  - https://spark.apache.org/docs/latest/tuning.html#garbage-collection-tuning
  - EMR : https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-plan-debugging.html
  - Tool to analyse the logs: https://gceasy.io/


[<<< Contents](#contents)

<a id="monitoring"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## Monitoring spark applications

  - Spark includes a configurable metrics system based on the [*dropwizard.metrics*](http://metrics.dropwizard.io/) library. It is set up via the Spark configuration. As we already are heavy users of **Graphite** and **Grafana**, we use the provided [Graphite sink](https://www.hammerlab.org/2015/02/27/monitoring-spark-with-graphite-and-grafana/).

    The Graphite sink needs to be **used with caution**. This is due to the fact that, for each metric, Graphite creates a fixed-size database to store data points. These zeroed-out [*Whisper*](https://graphite.readthedocs.io/en/latest/whisper.html) files consume quite a lot of disk space.

    By default, the application id is added to the namespace, which means that every *spark-submit* operation creates a new metric. Thanks to [SPARK-5847](https://issues.apache.org/jira/browse/SPARK-5847) it is now possible to **mitigate the \*Whisper\* behavior** and remove the *spark.app.id* from the namespace.

  ```
  	spark.metrics.namespace=$name
  ```

  
  - https://github.com/hammerlab/grafana-spark-dashboards
  - https://www.hammerlab.org/2015/02/27/monitoring-spark-with-graphite-and-grafana/
  - https://github.com/qubole/sparklens


[<<< Contents](#contents)

<a id="misc"></a>

----------------------------------------------------------------------------------------------------------------------------------------------------

## Misc 

* Try alternatives for AWS EMR with plain EC2 

  - https://github.com/nchammas/flintrock
  - https://heather.miller.am/blog/launching-a-spark-cluster-part-1.html


----------------------------------------------------------------------------------------------------------------------------------------------------

## Good reads

- https://medium.com/teads-engineering/lessons-learned-while-optimizing-spark-aggregation-jobs-f93107f7867f
- https://www.slideshare.net/databricks/apache-spark-coredeep-diveproper-optimization
- https://michalsenkyr.github.io/2018/01/spark-performance

Update the respective Gist for this notebook :)

https://gist.github.com/Mageswaran1989/e1957c887cd8ec900f84ae91842636b9

In [40]:
!jupyter nbconvert --to ltex SparkOptimization.ipynb

Traceback (most recent call last):
  File "/home/mageswarand/anaconda3/envs/vh/bin/jupyter-nbconvert", line 11, in <module>
    sys.exit(main())
  File "/home/mageswarand/anaconda3/envs/vh/lib/python3.6/site-packages/jupyter_core/application.py", line 270, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/home/mageswarand/anaconda3/envs/vh/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/mageswarand/anaconda3/envs/vh/lib/python3.6/site-packages/nbconvert/nbconvertapp.py", line 340, in start
    self.convert_notebooks()
  File "/home/mageswarand/anaconda3/envs/vh/lib/python3.6/site-packages/nbconvert/nbconvertapp.py", line 499, in convert_notebooks
    cls = get_exporter(self.export_format)
  File "/home/mageswarand/anaconda3/envs/vh/lib/python3.6/site-packages/nbconvert/exporters/base.py", line 113, in get_exporter
    % (name, ', '.join(get_export_names())))
ValueEr

In [27]:
! rm -rf e1957c887cd8ec900f84ae91842636b9
! git clone https://gist.github.com/Mageswaran1989/e1957c887cd8ec900f84ae91842636b9
! cd e1957c887cd8ec900f84ae91842636b9 && rm *
! cp SparkOptimization.html e1957c887cd8ec900f84ae91842636b9/SparkOptimization.html
! cd e1957c887cd8ec900f84ae91842636b9 && git add . && git commit -m "updated" && git push origin HEAD
! rm -rf e1957c887cd8ec900f84ae91842636b9

Cloning into 'e1957c887cd8ec900f84ae91842636b9'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 36 (delta 2), reused 25 (delta 1), pack-reused 9
Unpacking objects: 100% (36/36), done.


[master bedc1c1] updated
 21 files changed, 14003 insertions(+), 1592 deletions(-)
 delete mode 100644 JavaGarbageCollectionheap.png
 create mode 100644 SparkOptimization.html
 delete mode 100644 SparkOptimization.ipynb
 delete mode 100644 broadcat_explained.png
 delete mode 100644 bucketing.png
 delete mode 100644 cacahe.png
 delete mode 100644 coalesce.png
 delete mode 100644 dependencies.png
 delete mode 100644 driver_tasks.png
 delete mode 100644 explain.png
 delete mode 100644 null_counts.png
 delete mode 100644 paritionfilesize.png
 delete mode 100644 repartition.png
 delete mode 100644 repartition_memory.png
 delete mode 100644 schedule-process.png
 delete mode 100644 spark_execution_model.png
 delete mode 100644 spark_optimization_for_Advanced_users.ipynb
 delete mode 100644 spark_partition_file_size.png
 delete mode 100644 spill.png
 delete mode 100644 spill_corrected.png
 delete mode 100644 stage_execution_time.png
Counting objects: 3, done.
Delta compression using up to 12 t

[NbConvertApp] Converting notebook SparkOptimization.ipynb to html
[NbConvertApp] Writing 324153 bytes to SparkOptimization.html
